<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# FEC - Enrichissement du modèle de données

**Tags:** #fec #finance #snippet #operation #enrichiseement

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** Ce notebook permet d'enrichir le modèle de donnée FEC.

## Input

### Import libraries

In [ ]:
import re
import naas_data_product

### Setup Variables

In [ ]:
# Inputs
input_folder_path = "/home/ftp/FEC-engine/outputs/FEC/bdd_fin"

# Outputs
output_folder_path = "/home/ftp/FEC-engine/outputs/FEC/bdd_fin_enr"

## Model

### Récupération du dernier fichier input

In [ ]:
df_input = get_last_df(input_folder_path)
print("✅ Row fetched:", len(df_input))
df_input.head(1)

### Enrichissement de la base

In [ ]:
# Calcul résultat pour équilibrage bilan dans capitaux propre
db_rn = df_input.copy()

db_rn = db_rn[db_rn["COMPTE_NUM"].str.contains(r"^6|^7")]

to_group = ["ENTITY", "PERIOD"]
to_agg = {"VALUE": "sum"}
db_rn = db_rn.groupby(to_group, as_index=False).agg(to_agg)

db_rn["COMPTE_NUM"] = "10999999"
db_rn["RUBRIQUE_N3"] = "RESULTAT"

# Reorganisation colonne
to_select = ["ENTITY", "PERIOD", "COMPTE_NUM", "RUBRIQUE_N3", "VALUE"]
db_rn = db_rn[to_select]
db_rn

### Base de données FEC aggrégée avec variation

#### Aggrégation RUBRIQUE N3

In [ ]:
# Calcul var v = création de dataset avec Period_comp pour merge
db_var = df_input.copy()

# Regroupement
to_group = ["ENTITY", "PERIOD", "COMPTE_NUM", "RUBRIQUE_N3"]
to_agg = {"VALUE": "sum"}
db_var = db_var.groupby(to_group, as_index=False).agg(to_agg)

# Ajout des résultats au dataframe
db_var = pd.concat([db_var, db_rn], axis=0, sort=False)

# Creation colonne COMP
db_var["PERIOD_COMP"] = (db_var["PERIOD"].str[:4].astype(int) - 1).astype(str) + db_var[
    "PERIOD"
].str[-3:]

db_var

#### Création de la base comparable

In [ ]:
db_comp = db_var.copy()

# Suppression de la colonne période
db_comp = db_comp.drop("PERIOD_COMP", axis=1)

# Renommage des colonnes
to_rename = {"VALUE": "VALUE_N-1", "PERIOD": "PERIOD_COMP"}
db_comp = db_comp.rename(columns=to_rename)

db_comp.head(5)

#### Jointure des 2 tables et calcul des variations

In [ ]:
# Jointure entre les 2 tables
join_on = ["ENTITY", "PERIOD_COMP", "COMPTE_NUM", "RUBRIQUE_N3"]
db_var = (
    pd.merge(db_var, db_comp, how="left", on=join_on)
    .drop("PERIOD_COMP", axis=1)
    .fillna(0)
)

# Création colonne Var V
db_var["VARV"] = db_var["VALUE"] - db_var["VALUE_N-1"]

# Création colonne Var P (%)
db_var["VARP"] = db_var["VARV"] / db_var["VALUE_N-1"]

db_var

In [ ]:
db_cat = db_var.copy()

# Calcul des rubriques niveau 2
def rubrique_N2(row):
    numero_compte = str(row["COMPTE_NUM"])
    value = float(row["VALUE"])

    # BILAN SIMPLIFIE type IFRS NIV2

    to_check = ["^10", "^11", "^12", "^13", "^14"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "CAPITAUX_PROPRES"

    to_check = ["^15", "^16", "^17", "^18", "^19"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "DETTES_FINANCIERES"

    to_check = ["^20", "^21", "^22", "^23", "^25", "^26", "^27", "^28", "^29"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "IMMOBILISATIONS"

    to_check = ["^31", "^32", "^33", "^34", "^35", "^36", "^37", "^38", "^39"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "STOCKS"

    to_check = ["^40"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "DETTES_FOURNISSEURS"

    to_check = ["^41"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "CREANCES_CLIENTS"

    to_check = ["^42", "^43", "^44", "^45", "^46", "^47", "^48", "^49"]
    if any(re.search(x, numero_compte) for x in to_check):
        if value > 0:
            return "AUTRES_CREANCES"
        else:
            return "AUTRES_DETTES"

    to_check = ["^50", "^51", "^52", "^53", "^54", "^58", "^59"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "DISPONIBILITES"

    # COMPTE DE RESULTAT DETAILLE NIV2

    to_check = ["^60"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "ACHATS"

    to_check = ["^61", "^62"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "SERVICES_EXTERIEURS"

    to_check = ["^63"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "TAXES"

    to_check = ["^64"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "CHARGES_PERSONNEL"

    to_check = ["^65"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "AUTRES_CHARGES"

    to_check = ["^66"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "CHARGES_FINANCIERES"

    to_check = ["^67"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "CHARGES_EXCEPTIONNELLES"

    to_check = ["^68", "^78"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "AMORTISSEMENTS"

    to_check = ["^69"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "IMPOT"

    to_check = ["^70"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "VENTES"

    to_check = ["^71", "^72"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "PRODUCTION_STOCKEE_IMMOBILISEE"

    to_check = ["^74"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "SUBVENTIONS_D'EXPL."

    to_check = ["^75", "^791"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "AUTRES_PRODUITS_GESTION_COURANTE"

    to_check = ["^76", "^796"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "PRODUITS_FINANCIERS"

    to_check = ["^77", "^797"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "PRODUITS_EXCEPTIONNELS"

    to_check = ["^78"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "REPRISES_AMORT._DEP."

    to_check = ["^8"]
    if any(re.search(x, numero_compte) for x in to_check):
        return "COMPTES_SPECIAUX"


# Calcul des rubriques niveau 1
def rubrique_N1(row):
    categorisation = row.RUBRIQUE_N2

    # BILAN SIMPLIFIE type IFRS N1

    to_check = ["CAPITAUX_PROPRES", "DETTES_FINANCIERES"]
    if any(re.search(x, categorisation) for x in to_check):
        return "PASSIF_NON_COURANT"

    to_check = ["IMMOBILISATIONS"]
    if any(re.search(x, categorisation) for x in to_check):
        return "ACTIF_NON_COURANT"

    to_check = ["STOCKS", "CREANCES_CLIENTS", "AUTRES_CREANCES"]
    if any(re.search(x, categorisation) for x in to_check):
        return "ACTIF_COURANT"

    to_check = ["DETTES_FOURNISSEURS", "AUTRES_DETTES"]
    if any(re.search(x, categorisation) for x in to_check):
        return "PASSIF_COURANT"

    to_check = ["DISPONIBILITES"]
    if any(re.search(x, categorisation) for x in to_check):
        return "DISPONIBILITES"

    # COMPTE DE RESULTAT SIMPLIFIE N1

    to_check = ["ACHATS"]
    if any(re.search(x, categorisation) for x in to_check):
        return "COUTS_DIRECTS"

    to_check = [
        "SERVICES_EXTERIEURS",
        "TAXES",
        "CHARGES_PERSONNEL",
        "AUTRES_CHARGES",
        "AMORTISSEMENTS",
    ]
    if any(re.search(x, categorisation) for x in to_check):
        return "CHARGES_EXPLOITATION"

    to_check = ["CHARGES_FINANCIERES"]
    if any(re.search(x, categorisation) for x in to_check):
        return "CHARGES_FINANCIERES"

    to_check = ["CHARGES_EXCEPTIONNELLES", "IMPOT"]
    if any(re.search(x, categorisation) for x in to_check):
        return "CHARGES_EXCEPTIONNELLES"

    to_check = ["VENTES", "PRODUCTION_STOCKEE_IMMOBILISEE"]
    if any(re.search(x, categorisation) for x in to_check):
        return "CHIFFRE_D'AFFAIRES"

    to_check = [
        "SUBVENTIONS_D'EXPL.",
        "AUTRES_PRODUITS_GESTION_COURANTE",
        "REPRISES_AMORT._DEP.",
    ]
    if any(re.search(x, categorisation) for x in to_check):
        return "PRODUITS_EXPLOITATION"

    to_check = ["PRODUITS_FINANCIERS"]
    if any(re.search(x, categorisation) for x in to_check):
        return "PRODUITS_FINANCIERS"

    to_check = ["PRODUITS_EXCEPTIONNELS"]
    if any(re.search(x, categorisation) for x in to_check):
        return "PRODUITS_EXCEPTIONNELS"


# Calcul des rubriques niveau 0
def rubrique_N0(row):
    masse = row.RUBRIQUE_N1

    to_check = ["ACTIF_NON_COURANT", "ACTIF_COURANT", "DISPONIBILITES"]
    if any(re.search(x, masse) for x in to_check):
        return "ACTIF"

    to_check = ["PASSIF_NON_COURANT", "PASSIF_COURANT"]
    if any(re.search(x, masse) for x in to_check):
        return "PASSIF"

    to_check = [
        "COUTS_DIRECTS",
        "CHARGES_EXPLOITATION",
        "CHARGES_FINANCIERES",
        "CHARGES_EXCEPTIONNELLES",
    ]
    if any(re.search(x, masse) for x in to_check):
        return "CHARGES"

    to_check = [
        "CHIFFRE_D'AFFAIRES",
        "PRODUITS_EXPLOITATION",
        "PRODUITS_FINANCIERS",
        "PRODUITS_EXCEPTIONNELS",
    ]
    if any(re.search(x, masse) for x in to_check):
        return "PRODUITS"


# Mapping des rubriques
db_cat["RUBRIQUE_N2"] = db_cat.apply(lambda row: rubrique_N2(row), axis=1)
db_cat["RUBRIQUE_N1"] = db_cat.apply(lambda row: rubrique_N1(row), axis=1)
db_cat["RUBRIQUE_N0"] = db_cat.apply(lambda row: rubrique_N0(row), axis=1)


# Reorganisation colonne
to_select = [
    "ENTITY",
    "PERIOD",
    "COMPTE_NUM",
    "RUBRIQUE_N0",
    "RUBRIQUE_N1",
    "RUBRIQUE_N2",
    "RUBRIQUE_N3",
    "VALUE",
    "VALUE_N-1",
    "VARV",
    "VARP",
]
db_cat = db_cat[to_select]
db_cat

## Output

### Sauvegarde des fichiers en csv

In [ ]:
save_df(db_cat, output_folder_path)